In [6]:
#Python3(3.6) Notebook
# Please begin by running this cell
from classes import *

## DPLL 

The DPLL algorithm works as follows:

First, an initial unit propagation.

Then, repeatedly: 
1. Assign value to unassigned proposition
2. Unit propagate 
3. Backtrack whenever clause is violated
4. If complete assignment, satisfiable. 


Unit propagation assigns values to propositions that need to equal a certain value in order for the clause to satisfy. 

The below tree visualization shows a branch where 


Create trees via unassigned variables.  Visualization for the growth of the trees.  This builds on the unit propagation code; it takes in a sentence in CNF and returns a valid model of that sentence.

In [7]:
formula = to_cnf("( X & ( Y | Z ) )") 
print(formula)

(X & (Y | Z))


Notice how there is an unassigned clause but that unit propagation can not proceed. The next step is to arbitrarily assign one of the variables, keeping the record as a support, and proceed to see whether we have a model.

In [8]:

formula = to_cnf("( X & ( Y | Z ) )") 
print(formula)
(clauses, true_variables) = unit_propagation(str(formula))
print("True variables: " + ", ".join([str(variable) for variable in true_variables]))

(X & (Y | Z))
True variables: X


In [10]:
def assign_next(formula):
    for or_clause in formula.clauses:
        for literal in or_clause.literals:
            if literal.assignment is None:
                literal.assign(False)
                return 
            
formula = AND.from_string_to_cnf("(Y | Z) & (~Z | X) & Y")
print(formula)
print(formula.cnf_string())

assign_next(formula)
print(formula)

formula.unit_propagate_chris()
print(formula)

( ( Y | Z ) & ( ~Z | X ) & ( Y ) )
( ( Y | Z ) & ( ~Z | X ) & ( Y ) )
( ( Y = False | Z ) & ( ~Z | X ) & ( Y = False ) )


SyntaxError: invalid syntax (<string>, line 1)

On the other hand, see what happens when you need to backtrack!

In [ ]:
Y = L("Y")
Z = L("Z")
formula = AND(OR(L("X", True)), OR(Y, Z), OR(Y, NOT(Z)))

### Note on determining conflcits

Determining a minimal set of componenents involved in a conflict is, in general, NP-hard. Heuristics are often used, for instance in circuits, upstream components is often one heuristic. All components in the system is another (less useful) heuristic. Probabilistic mode estimation uses conflicts in order to generate the best possible guess about the health of all the components, if conflicts aren't particularly minimal (in the worst case, if they involve all components) Probabilistic Mode Estimation will only enumerate the failure space in a-priori failure probability order. So, it's not wrong to poorly minimize conflicts, it just makes probabilistic mode estimation perform poorly.   

## Probabilistic Mode Estimation

Probabalistic Mode Estimation or Mode Identification utilizing probabilistic inference's goal is to determine the state of the components in the system. 

One option, as always, is that all the components are operating correctly, but when there are discrepencies, heursitics can yield sets of components that could be at fault in the system. Mode Estimation then will give a probabilistically ordered list of configurations that your system could be in, instead. 

One way to perform this operation is constraint-based A*. 

The below tree visualization shows how the conflicts act to trim the search tree from paths that don't contain at least one of the conflicts. At each level of the tree is a decision to assign one of the components.

**Trim paths whose unknowns aren't present in at least one of the conflicts, or aren't present in all of the conflicts? That would depend on the means of generating conflicts. You might need to pre-process with a kernel diagnoses tool if you want to put more than one conflict through, though if it's _Trim paths whose unknowns aren't present in at least one of the conflicts_, you're probably fine and save computation time anyway.** _That said, each mode identification should include all the components that could be failed, so you'd want every conflict to be represented by one of the components marked failed in the identifcation, and trim once it became clear your choice of components wasn't going to satisfy one of the constraints (remaining assignments not in conflicts)._

The output of probabilistic mode estimation is a enumerable set of component assignments. 

In [ ]:
# Inputs: probabilities: Prior probabilities of component failures
#         conflicts: Heuristic-determined set of components that are connected to model-inconsistency
# Procedure: Perform constraint-based A*